In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
                                     
%matplotlib inline

In [2]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [3]:
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

In [4]:
#Mapeo los valores de nuestro analisis de datos para el Random Forest
postulations = pd.read_csv('../../DataAnalysis/postulaciones_nopostulaciones.csv')

In [5]:
postulations = DataFrameImputer().fit_transform(postulations)

In [6]:
postulations = postulations.iloc[:,3:]

In [7]:
postulations.head()

,nombre,estado,fechanacimiento,sexo,idpais,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,se_postulo
0,Doctorado,En Curso,1993-12-09,NO_DECLARA,1,Telecobrador - Importante Banco,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Call Center,1
1,Doctorado,Graduado,1986-08-21,MASC,1,Telecobrador - Importante Banco,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Call Center,1
2,Doctorado,En Curso,1998-03-14,FEM,1,Telecobrador - Importante Banco,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Call Center,1
3,Doctorado,Graduado,1981-12-31,FEM,1,Telecobrador - Importante Banco,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Call Center,1
4,Doctorado,Graduado,1989-05-05,FEM,1,Telecobrador - Importante Banco,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Call Center,1


In [8]:
# Mapping del postulante
postulations['fechanacimiento'] = datetime(postulations['fechanacimiento'],'%Y-%m-%d')
postulations['fechanacimiento'] = postulations['fechanacimiento'].map(lambda x: 2018 - x.year)

postulations.loc[ postulations['fechanacimiento'] <= 20, 'edadPostulante']= 0
postulations.loc[(postulations['fechanacimiento'] > 20) & (postulations['fechanacimiento'] <= 30), 'edadPostulante'] = 1
postulations.loc[(postulations['fechanacimiento'] > 30) & (postulations['fechanacimiento'] <= 40), 'edadPostulante'] = 2
postulations.loc[(postulations['fechanacimiento'] > 40) & (postulations['fechanacimiento'] <= 50), 'edadPostulante'] = 3
postulations.loc[(postulations['fechanacimiento'] > 50) & (postulations['fechanacimiento'] <= 60), 'edadPostulante'] = 4
postulations.loc[(postulations['fechanacimiento'] > 60) & (postulations['fechanacimiento'] <= 70), 'edadPostulante'] = 5
postulations.loc[(postulations['fechanacimiento'] > 70) & (postulations['fechanacimiento'] <= 80), 'edadPostulante'] = 6
postulations = DataFrameImputer().fit_transform(postulations)

In [9]:
postulations['nombre'] = pd.Categorical(postulations['nombre']).codes
postulations['estado'] = pd.Categorical(postulations['estado']).codes
postulations['sexo'] = pd.Categorical(postulations['sexo']).codes
postulations['nombre_zona'] = pd.Categorical(postulations['nombre_zona']).codes
postulations['tipo_de_trabajo'] = pd.Categorical(postulations['tipo_de_trabajo']).codes
postulations['nivel_laboral'] = pd.Categorical(postulations['nivel_laboral']).codes
postulations['nombre_area'] = pd.Categorical(postulations['nombre_area']).codes

In [10]:
#Dropeo algunos features
postulations = postulations.drop(['idpais','titulo','fechanacimiento'],axis = 1)

In [11]:
postulations.head()

,nombre,estado,sexo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,se_postulo,edadPostulante
0,0,1,3,3,1,4,25,1,1.0
1,0,2,2,3,1,4,25,1,2.0
2,0,1,1,3,1,4,25,1,0.0
3,0,2,1,3,1,4,25,1,2.0
4,0,2,1,3,1,4,25,1,1.0


In [ ]:
#Normalizamos los valores
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(postulations.iloc[:,:7]) 
postulations.iloc[:,:7] = scaled_values

In [12]:
postulations.to_csv('DataMappedMLP.csv',index = False)

In [13]:
test = pd.read_csv('../../test/100k_info_predecir.csv')

In [14]:
test.head()

,id,idaviso,idpostulante,nombre,estado,nombre_sort,fechanacimiento,sexo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,0,739260,6M9ZQR,Doctorado,Graduado,6.0,1976-03-29,FEM,1.0,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA
1,1,739260,6v1xdL,Terciario/Técnico,Graduado,2.0,1987-06-27,MASC,1.0,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA
2,2,739260,ezRKm9,Universitario,Graduado,3.0,1982-02-17,FEM,1.0,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA
3,3,758580,1Q35ej,Posgrado,Graduado,4.0,1949-12-24,MASC,1.0,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,NaN,Full-time,Otro,Tecnologia / Sistemas,TeleTech
4,4,758580,EAN4J6,Universitario,Abandonado,3.0,1986-05-27,FEM,1.0,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,NaN,Full-time,Otro,Tecnologia / Sistemas,TeleTech


In [ ]:
test = test.drop(['idpais','titulo','ciudad','mapacalle','denominacion_empresa','descripcion','nombre_sort'],axis = 1)

In [15]:
test['fechanacimiento'] = datetime(test['fechanacimiento'],'%Y-%m-%d')
test['fechanacimiento'] = test['fechanacimiento'].map(lambda x: 2018 - x.year)

test.loc[ test['fechanacimiento'] <= 20, 'edadPostulante']= 0
test.loc[(test['fechanacimiento'] > 20) & (test['fechanacimiento'] <= 30), 'edadPostulante'] = 1
test.loc[(test['fechanacimiento'] > 30) & (test['fechanacimiento'] <= 40), 'edadPostulante'] = 2
test.loc[(test['fechanacimiento'] > 40) & (test['fechanacimiento'] <= 50), 'edadPostulante'] = 3
test.loc[(test['fechanacimiento'] > 50) & (test['fechanacimiento'] <= 60), 'edadPostulante'] = 4
test.loc[(test['fechanacimiento'] > 60) & (test['fechanacimiento'] <= 70), 'edadPostulante'] = 5
test.loc[(test['fechanacimiento'] > 70) & (test['fechanacimiento'] <= 80), 'edadPostulante'] = 6
test = DataFrameImputer().fit_transform(test)
test= test.drop(['fechanacimiento'],axis=1)

In [16]:
test['nombre'] = pd.Categorical(test['nombre']).codes
test['estado'] = pd.Categorical(test['estado']).codes
test['sexo'] = pd.Categorical(test['sexo']).codes
test['nombre_zona'] = pd.Categorical(test['nombre_zona']).codes
test['tipo_de_trabajo'] = pd.Categorical(test['tipo_de_trabajo']).codes
test['nivel_laboral'] = pd.Categorical(test['nivel_laboral']).codes
test['nombre_area'] = pd.Categorical(test['nombre_area']).codes

In [18]:
test.head()

,id,idaviso,idpostulante,nombre,estado,nombre_sort,sexo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,edadPostulante
0,0,739260,6M9ZQR,0,2,6.0,0,Nuestro cliente es una empresa nac. ubicada en...,1,1,1,29,3.0
1,1,739260,6v1xdL,5,2,2.0,1,Nuestro cliente es una empresa nac. ubicada en...,1,1,1,29,2.0
2,2,739260,ezRKm9,6,2,3.0,0,Nuestro cliente es una empresa nac. ubicada en...,1,1,1,29,2.0
3,3,758580,1Q35ej,3,2,4.0,1,"\r\nTeleTech, the worlwide leader in outsource...",1,1,3,164,5.0
4,4,758580,EAN4J6,6,0,3.0,0,"\r\nTeleTech, the worlwide leader in outsource...",1,1,3,164,2.0


In [ ]:
#Normalizamos los valores
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(test.iloc[:,1:]) 
test.iloc[:,1:] = scaled_values

In [20]:
test.to_csv('TestMappedMLP.csv',index = False)